In [877]:
import inspect
import random
from weakref import WeakKeyDictionary

In [936]:
class Symbol(object):
    def __init__(self, values):
        self.parent = None
        self.InitVals(values)
        
    def InitVals(self, values):
        # set all of my potential values
        
        # allow synyax shorthand 
        # symbol = Symbol(Fruit) = Symbol([Apple, Pear])
        if inspect.isclass(values):
            self.values = []
            for subclass in values.__subclasses__():
                self.values.append(subclass())
        else:
            # TODO: ensure iterable
            self.values = values
        
        # initialize state to superposition
        self._value = None
        
        # expose all of my possible values' attributes as if they were my own
        self.CalcAttributes()
        
    def __repr__(self):
        return str(self.val)
    
    def __getattr__(self, attr):
        
        # allow self.val to inspect without collapsing my value
        if attr == 'val':
            if self._value:
                return self._value
            else:
                
                if len(self.values) == 1:
                    return self.values[0]
                else:
                    return self.values
        raise AttributeError("{} object has no attribute {}".format(self.__class__, attr))
    
    def Observe(self):
        if self._value:
            return self._value
        else:
            return self.Collapse()
    
    def Collapse(self, new_value=None):
        
        # set value
        if new_value:
            
            # TODO: restrict to all values consistent with new_value
            self._value = new_value
        else:
            self._value = random.choice(self.values)
        
        # now I only have one value
        self.values = [self._value]
        
        # propagate the change upwards: everything above me must be consistent
        # with me having this value.
        if self.parent:
            self.parent.Restrict(self, self._value)

        # update my attributes
        self.CalcAttributes()
        
        return self._value
    
    def CalcAttributes(self):
        
        # expose the attributes of all my potential values
        attribute_values = {}
        for obj in self.values:
            
            # list all of the object's public values
            attributes = [i for i in dir(obj) if i[0] != '_' and not callable(getattr(obj, i))]
            
            # add all of obj.attribute to the set of possible values
            for attribute in attributes:
                
                # initialize dict
                if attribute not in attribute_values:
                    attribute_values[attribute] = []
                
                # get obj.val
                val = obj.__getattribute__(attribute)
                
                # if obj.val is a symbol, i must be consistent with all of its values
                if isinstance(val, Symbol):
                    attribute_values[attribute].extend(val.values)
                else:
                    # TODO: check if iterable
                    attribute_values[attribute].append(val)
        
        # add these attributes to myself.
        for key, vals in attribute_values.items():
                symbol = Symbol(vals)
                symbol.parent = self
                
                # TODO: check if i already have the attribute first
                setattr(self, key, symbol)

    def Restrict(self, child, newval):
        # change content of values to be consistent with new value for child
        # Restriction is:
        # now that we know the specific value of this property, which of my values are consistent with that?
        
        # determine the attribute
        for key, val in self.__dict__.items():
            if val == child:
                attr = key
                break
        
        # check each of my potential values to see if they are consistent with having
        # obj.attr = newval
        new_values = []
        for obj in self.values:
            
            # if the object doesnt even have this attribute, forget it
            if not hasattr(obj, attr):
                continue
            
            # get the object's obj.attr
            obj_attr_val =  getattr(obj, attr)
            
            # if is obj.attr is symbol, how can we make sure it is consistent
            # with obj.attr = newval?
            # nested symbols
            if isinstance(obj_attr_val, Symbol):
                
                # obj.attr = Symbol ⊃ newval
                if newval in obj_attr_val.values:
                    obj_attr_val.Collapse(new_value=newval)
                    new_values.append(obj)
                    continue
            else:
                # if obj.attr is not symbol, check that obj.attr is consistent with newval
                if obj_attr_val == newval:
                    new_values.append(obj)
                    continue
        
        # big problem here if true
        if len(new_values) == 0:
            raise Exception('empty restriction!')
            
        self.values = new_values
        
        # if we've restricted to a single value, might as well collapse now.
        if len(new_values) == 1:
            self.Collapse()
            

# TODO

values = [v1, v2, v3]

## Collapse

1. person.item collapses.

person.item -> Sword

person.item = [Sword].


2. we need person to be holding a weapon in general. 

person.item = all subclasses of Item.
Weapon is a subclass of Item, so this is consistent:
anything we could have done with person.item, we can do after it collapses to
Weapon.

person.item -> Weapon

Weapon is still generic: its subclasses include Sword and Axe.

person.item = [Sword, Axe]

new_value: Weapon
the requirement here for each v1 is that new_values ⊃ v1, where ⊃ is defined
for class / subclass relations


3. Collapse to a Symbol.

we restrict to self.values -> self.values ∩ new_value.values

4. Collapse to a list / set / iterable

In general, if new_value is array-like, self.values -> self.values ∩ new_value

If the new_value contains things not contained by v1 (collapse person.weapon to Item, we remain at person.weapon)
we need to ensure we are compatible with original spec of my object.

If v1 contains more things than new_value, we need to ensure we are compatible with new_value


## Restriction
What does it mean for a potential value to be consistent with this attribute value?

due to collapse, i know that my self.attr = new_value
if v1.attr == new_value, good

if v1.attr is the same type as new_value?

if type of v1.attr is subclass of type of new_value?

In [949]:
Apple in Fruit.__subclasses__()

True

In [904]:
class Symbolic(object):
    def __init__(self):
        self.symbols = WeakKeyDictionary()
 
    def __get__(self, instance_obj, objtype):
#         return self.symbols[instance_obj].Ref()
        return self.symbols[instance_obj]
 
    def __set__(self, instance, values):
        self.symbols[instance] = Symbol(values)
 
    def __delete__(self, instance):
        del self.symbols[instance]

        
class Fruit(object):
    color = Symbolic()
    def __init__(self, name):
        self.name = name
    def method(self):
        print('method man')

class Apple(Fruit):
    def __init__(self):
        self.name = 'Apple'
        self.color = ['red', 'green']

class Pear(Fruit):
    def __init__(self):
        self.name = 'Pear'
        self.color = ['red', 'yellow']

        
class State(object):
    fruit = Symbolic()
    def __init__(self):
        self.fruit = Fruit

In [937]:
state = State()

In [938]:
print(state.fruit.color)

['red', 'green', 'red', 'yellow']


In [939]:
state.fruit

[<__main__.Apple object at 0x104e44f28>, <__main__.Pear object at 0x104e44da0>]

In [940]:
state.fruit.name

['Apple', 'Pear']

In [941]:
state.fruit.Observe()

In [942]:
state.fruit.name

Pear

# Questions

1. how to naturally handle references to symbols

    a. what is the use case for symbols? use in text?
    
2. do nested symbols work?

3. type vs. instance in restriction

    restrict self.attr to value
    if values[1].attr is value, and symbol.values[2].attr is an instance of the same class as value,
    then we should allow either?
    it should be instead a relation of subclasses

4. when recalcing attributes, order matters:
    
    if i recalculate my attributes before a child symbol, i wont catch all of the child symbol's properties
    but do i want to do this? consider

```
A
|
|---|
B   C
   / \
   D  E
```

A should only expose the the attributes of its immediate values.

```
A: <B, C>

A.a
B.b
C.c
C.d

recalc:
A.a
A.b
A.c
A.d

Nested:
A.c.z ? 
possible or no?
only if z is a property of all classes of c (superclass property, etc)

```

We should be adopting attributes only if they belong to the class -> superclass?
some sort of class spec for symbol.values?
Restriction is:
now that we know the specific value of this property, which of my values are consistent with that?

In [896]:
state.fruit